In [192]:
import pandas as pd
df= pd.read_csv(r"C:\Users\Access\Documents\AI ROBOT\XAUUSD_Candlestick_1_M_BID_2020-1-1_2024-12-31.csv")

In [193]:
df=df.drop(columns='Volume')

In [194]:
import pandas as pd

# تحويل العمود الزمني إلى نوع datetime وتعيينه كمؤشر
df['Gmt time'] = pd.to_datetime(df['Gmt time'])
df.set_index('Gmt time', inplace=True)

# تحويل البيانات من 1 دقيقة إلى 5 دقائق
df_5m = df.resample('5T').agg({
    'Open': 'first',   # أول سعر افتتاح في الفاصل الزمني
    'High': 'max',     # أعلى سعر في الفاصل الزمني
    'Low': 'min',      # أقل سعر في الفاصل الزمني
    'Close': 'last'    # آخر سعر إغلاق في الفاصل الزمني
})

# عرض النتيجة
print(df_5m)

C:\Users\Access\AppData\Local\Temp\ipykernel_2940\422809626.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_5m = df.resample('5T').agg({


                         Open      High       Low     Close
Gmt time                                                   
2020-01-01 00:00:00  1516.915  1516.915  1516.915  1516.915
2020-01-01 00:05:00  1516.915  1516.915  1516.915  1516.915
2020-01-01 00:10:00  1516.915  1516.915  1516.915  1516.915
2020-01-01 00:15:00  1516.915  1516.915  1516.915  1516.915
2020-01-01 00:20:00  1516.915  1516.915  1516.915  1516.915
...                       ...       ...       ...       ...
2024-12-31 23:35:00  2624.255  2624.255  2624.255  2624.255
2024-12-31 23:40:00  2624.255  2624.255  2624.255  2624.255
2024-12-31 23:45:00  2624.255  2624.255  2624.255  2624.255
2024-12-31 23:50:00  2624.255  2624.255  2624.255  2624.255
2024-12-31 23:55:00  2624.255  2624.255  2624.255  2624.255

[526176 rows x 4 columns]


In [195]:
df_5m=df_5m.reset_index(drop=False) 

In [196]:
df_5m

,Gmt time,Open,High,Low,Close
0,2020-01-01 00:00:00,1516.915,1516.915,1516.915,1516.915
1,2020-01-01 00:05:00,1516.915,1516.915,1516.915,1516.915
2,2020-01-01 00:10:00,1516.915,1516.915,1516.915,1516.915
3,2020-01-01 00:15:00,1516.915,1516.915,1516.915,1516.915
4,2020-01-01 00:20:00,1516.915,1516.915,1516.915,1516.915
...,...,...,...,...,...
526171,2024-12-31 23:35:00,2624.255,2624.255,2624.255,2624.255
526172,2024-12-31 23:40:00,2624.255,2624.255,2624.255,2624.255
526173,2024-12-31 23:45:00,2624.255,2624.255,2624.255,2624.255
526174,2024-12-31 23:50:00,2624.255,2624.255,2624.255,2624.255


In [138]:
import pandas as pd

df=df_5m

# تعريف دالة التحقق من الشمعة الكبيرة مقارنة بالظلال
def is_large_body(candle):
    body = abs(candle['Open'] - candle['Close'])
    lower_shadow = abs(candle['Low'] - min(candle['Open'], candle['Close']))
    upper_shadow = abs(candle['High'] - max(candle['Open'], candle['Close']))
    return body > (lower_shadow + upper_shadow)

# تعريف دالة التحقق من شمعة الهامر
def is_hammer(candle):
    body = abs(candle['Open'] - candle['Close'])
    lower_shadow = abs(candle['Low'] - min(candle['Open'], candle['Close']))
    upper_shadow = abs(candle['High'] - max(candle['Open'], candle['Close']))
    return lower_shadow > 2 * body and upper_shadow < body

# دالة حساب صفقة شراء باستخدام الهامر
def calculate_buy_trade_hammer(row, df, tp_multiplier=1.5, sl_multiplier=1):
    idx = row.name  # الفهرس الزمني (Timestamp)
    if idx == df.index[0]:  # إذا كانت الشمعة الأولى، لا توجد صفقة
        return 0, None, None
    first_candle = df.loc[df.index[df.index.get_loc(idx) - 1]]  # الشمعة السابقة
    second_candle = row  # الشمعة الحالية
    if is_large_body(first_candle) and first_candle['Close'] < first_candle['Open'] and is_hammer(second_candle):
        points = abs(first_candle['Close'] - first_candle['Low'])
        take_profit = second_candle['Close'] + (points * tp_multiplier)
        stop_loss = second_candle['Low'] - (points * sl_multiplier)
        return 1, take_profit, stop_loss
    return 0, None, None

# تطبيق الأعمدة الجديدة للصفقات الشرائية
df['buy_trade_hammer'], df['tp_buy_hammer'], df['sl_buy_hammer'] = zip(*df.apply(lambda row: calculate_buy_trade_hammer(row, df), axis=1))

# تحديد نوع الصفقة للشراء
df['trade_type_hammer'] = df.apply(lambda row: 1 if row['buy_trade_hammer'] == 1 else 0, axis=1)

# دمج أعمدة TP و SL للصفقات الشرائية
df['tp_hammer'] = df.apply(lambda row: row['tp_buy_hammer'] if row['trade_type_hammer'] == 1 else None, axis=1)
df['sl_hammer'] = df.apply(lambda row: row['sl_buy_hammer'] if row['trade_type_hammer'] == 1 else None, axis=1)

# دالة لتحديد حالة الصفقة الشرائية
def determine_trade_result_hammer(row, df):
    idx = row.name  # الفهرس الزمني (Timestamp)
    if row['trade_type_hammer'] == 1:  # صفقة شراء
        for i in range(df.index.get_loc(idx) + 1, len(df)):
            future_row = df.iloc[i]
            if future_row['High'] >= row['tp_hammer']:
                return 2  # رابحة
            elif future_row['Low'] <= row['sl_hammer']:
                return 1  # خاسرة
    return 0  # لا توجد نتيجة

# تطبيق دالة تحديد حالة الصفقة الشرائية
df['trade_result_hammer'] = df.apply(lambda row: determine_trade_result_hammer(row, df), axis=1)

# عرض الأعمدة المطلوبة للصفقات الشرائية
print(df[['Gmt time','Open', 'High', 'Low', 'Close', 'trade_type_hammer', 'trade_result_hammer', 'tp_hammer', 'sl_hammer']])

                  Gmt time      Open      High       Low     Close  \
0      2020-01-01 00:00:00  1516.915  1516.915  1516.915  1516.915   
1      2020-01-01 00:05:00  1516.915  1516.915  1516.915  1516.915   
2      2020-01-01 00:10:00  1516.915  1516.915  1516.915  1516.915   
3      2020-01-01 00:15:00  1516.915  1516.915  1516.915  1516.915   
4      2020-01-01 00:20:00  1516.915  1516.915  1516.915  1516.915   
...                    ...       ...       ...       ...       ...   
526171 2024-12-31 23:35:00  2624.255  2624.255  2624.255  2624.255   
526172 2024-12-31 23:40:00  2624.255  2624.255  2624.255  2624.255   
526173 2024-12-31 23:45:00  2624.255  2624.255  2624.255  2624.255   
526174 2024-12-31 23:50:00  2624.255  2624.255  2624.255  2624.255   
526175 2024-12-31 23:55:00  2624.255  2624.255  2624.255  2624.255   

        trade_type_hammer  trade_result_hammer  tp_hammer  sl_hammer  
0                       0                    0        NaN        NaN  
1                

In [188]:
df_filter = df[df['trade_duration_hammer'].notna()]

In [189]:
df_filter

,Gmt time,Open,High,Low,Close,buy_trade_hammer,tp_buy_hammer,sl_buy_hammer,trade_type_hammer,tp_hammer,sl_hammer,trade_result_hammer,trade_duration_hammer
310,2020-01-02 01:50:00,1519.300,1519.408,1518.875,1519.365,1,1519.3650,1518.875,1,1519.3650,1518.875,2,1
399,2020-01-02 09:15:00,1519.995,1520.265,1519.515,1520.205,1,1520.2350,1519.495,1,1520.2350,1519.495,2,1
496,2020-01-02 17:20:00,1524.685,1524.685,1522.715,1524.605,1,1525.0055,1522.448,1,1525.0055,1522.448,2,1
718,2020-01-03 11:50:00,1550.038,1550.138,1549.355,1549.818,1,1549.8975,1549.302,1,1549.8975,1549.302,2,1
765,2020-01-03 15:45:00,1548.375,1548.388,1547.828,1548.288,1,1548.3330,1547.798,1,1548.3330,1547.798,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
525855,2024-12-30 21:15:00,2606.975,2607.245,2606.535,2607.185,1,2607.3950,2606.395,1,2607.3950,2606.395,2,1
525911,2024-12-31 01:55:00,2609.065,2609.105,2608.025,2608.745,1,2608.9550,2607.885,1,2608.9550,2607.885,2,1
525934,2024-12-31 03:50:00,2604.704,2604.704,2603.305,2604.498,1,2605.6515,2602.536,1,2605.6515,2602.536,2,9
525959,2024-12-31 05:55:00,2604.345,2604.405,2602.475,2604.235,1,2604.8155,2602.088,1,2604.8155,2602.088,2,1


In [184]:
df.value_counts("trade_duration_hammer")

trade_duration_hammer
1      2874
2       513
3       179
4        86
5        51
6        31
13       16
7        15
8        11
10       11
9        10
16        6
12        5
14        5
15        4
21        3
11        3
45        2
26        2
19        2
22        1
23        1
27        1
32        1
37        1
44        1
17        1
108       1
110       1
121       1
Name: count, dtype: int64

In [140]:
df.value_counts("trade_result_hammer")

trade_result_hammer
0    522337
2      3046
1       793
Name: count, dtype: int64

In [141]:
df_ = df[['Gmt time','Open', 'High', 'Low', 'Close', 'trade_type_hammer', 'trade_result_hammer', 'tp_hammer', 'sl_hammer']]

In [177]:
df_

,Gmt time,Open,High,Low,Close,trade_type_hammer,trade_result_hammer,tp_hammer,sl_hammer
0,2020-01-01 00:00:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
1,2020-01-01 00:05:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
2,2020-01-01 00:10:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
3,2020-01-01 00:15:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
4,2020-01-01 00:20:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
526171,2024-12-31 23:35:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN
526172,2024-12-31 23:40:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN
526173,2024-12-31 23:45:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN
526174,2024-12-31 23:50:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN


In [178]:
df_.to_csv(r"C:\Users\Access\Documents\AI ROBOT\XAUUSD_5m_trade_result_hammer_trade_list.csv", index=True)

In [163]:
import pandas as pd

df = df_.copy()

# تصفية الصفقات التي تحتوي على نتائج
df = df[df['trade_result_hammer'] != 0]

# استخراج التاريخ فقط
df['Gmt date'] = df['Gmt time'].dt.date

# تصنيف الصفقات
df['trade_type'] = df['trade_result_hammer'].apply(lambda x: 'رابحة' if x == 2 else 'خاسرة')

# حساب عدد الصفقات لكل يوم
daily_trades = df.groupby(['Gmt date', 'trade_type']).size().unstack(fill_value=0)

# إعادة تسمية الأعمدة
daily_trades = daily_trades.rename(columns={'رابحة': 'Winning Trades', 'خاسرة': 'Losing Trades'})

# إنشاء نطاق تواريخ شامل من البيانات الأصلية
start_date = df_['Gmt time'].min().normalize()
end_date = df_['Gmt time'].max().normalize()
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# تحويل الفهرس إلى تنسيق تاريخي
daily_trades.index = pd.to_datetime(daily_trades.index)

# إعادة الفهرس ليشمل جميع التواريخ
daily_trades = daily_trades.reindex(all_dates, fill_value=0)

# تصفية أيام السبت (5) والأحد (6) باستخدام weekday
daily_trades = daily_trades[daily_trades.index.weekday < 5]  # 0-4 = من الاثنين إلى الجمعة

# إضافة عمود الصفقات الكلية
daily_trades['Total Trades'] = daily_trades['Winning Trades'] + daily_trades['Losing Trades']

# إعادة تنسيق الفهرس لعرضه كتواريخ بدون وقت
daily_trades.index = daily_trades.index.date

print(daily_trades)

trade_type  Losing Trades  Winning Trades  Total Trades
2020-01-01              0               0             0
2020-01-02              0               3             3
2020-01-03              0               2             2
2020-01-06              1               1             2
2020-01-07              0               5             5
...                   ...             ...           ...
2024-12-25              0               0             0
2024-12-26              0               3             3
2024-12-27              1               3             4
2024-12-30              0               6             6
2024-12-31              1               3             4

[1305 rows x 3 columns]


In [164]:
daily_trades[:60]

trade_type,Losing Trades,Winning Trades,Total Trades
2020-01-01,0,0,0
2020-01-02,0,3,3
2020-01-03,0,2,2
2020-01-06,1,1,2
2020-01-07,0,5,5
2020-01-08,0,3,3
2020-01-09,0,4,4
2020-01-10,0,5,5
2020-01-13,0,2,2
2020-01-14,0,5,5


In [176]:
daily_trades.to_csv(r"C:\Users\Access\Documents\AI ROBOT\XAUUSD_5m_trade_result_hammer.csv", index=True)

In [166]:
daily_trades.describe().round(1).drop(['count','std'])

trade_type,Losing Trades,Winning Trades,Total Trades
mean,0.6,2.3,2.9
min,0.0,0.0,0.0
25%,0.0,1.0,2.0
50%,0.0,2.0,3.0
75%,1.0,3.0,4.0
max,4.0,8.0,10.0


In [173]:
# حساب التكرارات والنسب المئوية
total_counts = daily_trades['Total Trades'].value_counts()
total_percent = daily_trades['Total Trades'].value_counts(normalize=True) * 100

# دمج النتائج في DataFrame
result_df = pd.DataFrame({
    'Count': total_counts,
    'Percentage (%)': total_percent.round(1)
})

result_df['Percentage (%)'] = result_df['Percentage (%)'].astype(str) + '%'

# طباعة النتيجة
print(result_df)

              Count Percentage (%)
Total Trades                      
2               284          21.8%
3               258          19.8%
4               212          16.2%
1               207          15.9%
5               115           8.8%
0               106           8.1%
6                71           5.4%
7                35           2.7%
8                11           0.8%
9                 5           0.4%
10                1           0.1%


In [171]:
# حساب التكرارات والنسب المئوية
winning_counts = daily_trades['Winning Trades'].value_counts()
winning_percent = daily_trades['Winning Trades'].value_counts(normalize=True) * 100

# دمج النتائج في DataFrame
result_df = pd.DataFrame({
    'Count': winning_counts,
    'Percentage (%)': winning_percent.round(1)
})

result_df['Percentage (%)'] = result_df['Percentage (%)'].astype(str) + '%'

# طباعة النتيجة
print(result_df)

                Count Percentage (%)
Winning Trades                      
2                 330          25.3%
1                 280          21.5%
3                 247          18.9%
0                 164          12.6%
4                 159          12.2%
5                  78           6.0%
6                  30           2.3%
7                  11           0.8%
8                   6           0.5%


In [170]:
# حساب التكرارات والنسب المئوية
losing_counts = daily_trades['Losing Trades'].value_counts().sort_index()
losing_percent = daily_trades['Losing Trades'].value_counts(normalize=True).sort_index() * 100

# دمج النتائج في DataFrame
result_df = pd.DataFrame({
    'Count': losing_counts,
    'Percentage (%)': losing_percent.round(1)
})

result_df['Percentage (%)'] = result_df['Percentage (%)'].astype(str) + '%'

# طباعة النتيجة
print(result_df)

               Count Percentage (%)
Losing Trades                      
0                729          55.9%
1                408          31.3%
2                142          10.9%
3                 22           1.7%
4                  4           0.3%
